In [70]:
import get_data as get_data
import load_data as load

import pandas as pd
import seaborn as sns
from pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

In [71]:
data = pd.read_csv('./data_research/dump_for_Jakob.csv', names=\
                   ['exchange', 'pair', 'model', 'prediction_close', 'prediction_trend',
                   'prediction_percentage_trend', 'date', 'real_close', 'real_trend',
                   'real_percentage_trend', 'close_diff', 'trend_diff',
                   'trend_percentage_diff', 'percentage_diff',
                   'prediction_close_on_precentage'])

frame = data[['date', 'prediction_close', 'real_close', 'model', 'pair', 'prediction_trend']]
frame = frame.dropna()
frame = frame.reset_index()
frame = frame.dropna()[['prediction_close','real_close', 'date','pair' ,'model','prediction_trend']]

In [109]:
data[['date', 'prediction_close', 'real_close', 'model', 'pair', 'prediction_trend']].tail()

,date,prediction_close,real_close,model,pair,prediction_trend
582,2017-07-26,0.261646,0.17259,LSTMCloseFeature,USDT_XRP,0.000211
583,2017-07-26,0.261646,0.17259,LSTM5Features,USDT_XRP,0.000211
584,2017-07-26,0.258063,0.17259,LSTMHighLow,USDT_XRP,-0.000769
585,2017-07-26,0.258063,0.17259,LSTMHighLowRaw,USDT_XRP,-0.000769
586,2017-07-26,0.255564,0.17259,LSTMRSIVariability,USDT_XRP,0.001488


In [98]:
LSTMCloseFeature = frame[frame.model == 'LSTMCloseFeature'].copy()
LSTM5Features = frame[frame.model == 'LSTM5Features'].copy()
LSTMHighLow  = frame[frame.model == 'LSTMHighLow'].copy()
LSTMHighLowRaw = frame[frame.model == 'LSTMHighLowRaw'].copy()
Atr14Moment20 = frame[frame.model == 'Atr_14moment_20'].copy()
Atr14Moment20LowHigh = frame[frame.model == 'Atr_14moment_20lowhigh'].copy()
Macd_12_26_9stoch_d_14_3roc_14 = frame[frame.model == 'Macd_12_26_9stoch_d_14_3roc_14'].copy()
Moment_20ema_rel_diff_10ma_rel_diff_50 = frame[frame.model == 'Moment_20ema_rel_diff_10ma_rel_diff_50'].copy()
Roc_14moment_20ema_rel_diff_10 = frame[frame.model == 'Roc_14moment_20ema_rel_diff_10'].copy()

In [120]:
model_names = [
    LSTMCloseFeature,
    LSTM5Features,
    LSTMHighLow,
    LSTMHighLowRaw,

    Atr14Moment20,######!!!!!
    Atr14Moment20LowHigh,#######!!!!!
    Macd_12_26_9stoch_d_14_3roc_14,######
    Moment_20ema_rel_diff_10ma_rel_diff_50,
    Roc_14moment_20ema_rel_diff_10

]

In [116]:
model_names = [
#    LSTMCloseFeature,
#    LSTM5Features,
#    LSTMHighLow,
#    LSTMHighLowRaw,

    Atr14Moment20,######!!!!!
    Atr14Moment20LowHigh,#######!!!!!
    Macd_12_26_9stoch_d_14_3roc_14,######
    Moment_20ema_rel_diff_10ma_rel_diff_50,
    Roc_14moment_20ema_rel_diff_10

]
model_results = []

for model in model_names:
    pr_model = pd.DataFrame()
    for i in frame.pair.unique():
        a =  model[model.pair == i].copy()
        a ['pr_label'] = (a ['prediction_close'] > a['prediction_close'].shift(-1)).astype('int') 
        a['real_label'] = (a['real_close'] > a['real_close'].shift(-1)).astype('int')  
        pr_model =pr_model.append(a.copy())
    model_results.append(pr_model.reset_index()) 
    
    
for i in range(len(model_names)):
    model_names[i] = model_results[i]

result_voting = []
res_close = []
for j in ['USDT_BTC']: # Когда будут предсказания для всех типов рынка - не только для BTC
                       # добавляем через frame.pair.unique()
    a = 0  
    z = 0
    for i in range(len(model_names)):
        a+= model_names[i][model_names[i].pair == j].pr_label.values.reshape(-1,1)
       
        z += model_names[i][model_names[i].pair == j].prediction_close.shift(-1).values.reshape(-1,1) 
        #z += model_names[i][model_names[i].pair == j].prediction_trend.values.reshape(-1,1)

    result_voting.append(a)
    res_close.append(z)

result_voting = np.array(result_voting).reshape(-1,)

probability = np.array(result_voting / len(model_names))
VOTING = np.round(probability)

TRUE_LABELS  = model_names[0][model_names[0].pair == "USDT_BTC"].real_label.values
print ('number model: ', len(model_names))
print (result_voting)
print ('probability: ' , probability*100, '%')
print ('VOTE: ' ,VOTING)
print (TRUE_LABELS)

np.array(VOTING == TRUE_LABELS).astype('int').sum()/VOTING.shape[0] *100

number model:  5
[5 1 4 4 5 4 4 4 4 4 2 3 1 2 0]
probability:  [ 100.   20.   80.   80.  100.   80.   80.   80.   80.   80.   40.   60.
   20.   40.    0.] %
VOTE:  [ 1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  0.]
[1 1 1 0 1 1 1 0 0 1 1 0 1 1 0]


46.666666666666664

In [123]:
quality = pd.DataFrame({'model': [], 'quality': []})

for i in range(len(model_names)):

    quality_m = (model_names[i].real_label == model_names[i].pr_label)\
                .astype('int').sum()/model_names[i].shape[0]
        
    name_m = list(model_names[i].model.unique())
    print (pd.DataFrame(data = {'model': name_m, 'quality':quality_m }))
    quality = quality.append (pd.DataFrame(data = {'model': name_m, 'quality':quality_m }))

AttributeError: 'DataFrame' object has no attribute 'real_label'

In [125]:
model_results[0]

,index,prediction_close,real_close,date,pair,model,prediction_trend,pr_label,real_label
0,5,4134.551270,4051.697404,2017-08-13,USDT_BTC,Atr_14moment_20,520.304932,1,1
1,40,3614.246338,3863.500000,2017-08-12,USDT_BTC,Atr_14moment_20,16.537109,1,1
2,75,3597.709229,3641.337100,2017-08-11,USDT_BTC,Atr_14moment_20,96.892090,1,1
3,197,3120.575195,3229.000000,2017-08-06,USDT_BTC,Atr_14moment_20,170.517578,1,0
4,227,2950.057617,3256.536656,2017-08-05,USDT_BTC,Atr_14moment_20,5.008545,1,1
5,257,2945.049072,2863.009999,2017-08-04,USDT_BTC,Atr_14moment_20,-1.153809,0,1
6,287,2946.202881,2794.890000,2017-08-03,USDT_BTC,Atr_14moment_20,-7.850098,0,1
7,317,2954.052979,2699.000000,2017-08-02,USDT_BTC,Atr_14moment_20,15.881104,1,0
8,347,2938.171875,2736.482685,2017-08-01,USDT_BTC,Atr_14moment_20,41.434570,1,0
9,377,2896.737305,2855.000000,2017-07-31,USDT_BTC,Atr_14moment_20,43.873535,1,1


In [ ]:
plt.plot(np.array(res_close).reshape(-1,)/4)
model_names[0][model_names[0].pair == 'USDT_BTC'].real_close.plot()